In [ ]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook
import os
import re
import matplotlib.pyplot as plt
from notebook.notebookapp import main

# Set working directory
os.chdir('/Users/sousekilyu/Documents/GitHub/GaoKaoVer3')

# Define the major lists
majorList = [
    ["新闻|传播", "新闻传播学"],
    ["法学|法律", "法学"],
    ["计算机", "计算机类"],
    ["软件", "软件工程"],
    ["土木", "土木工程类"],
    ["数据科学与大数据技术", "数据科学与大数据技术"],
    ["自然保护与环境生态|环境生态", "环境生态类"],
    ["轨道交通电气与控制", "轨道交通电气类"],
    ["旅游管理", "旅游管理"]
]

majorList2 = [
    ["新闻|传播|广告|出版", "新闻传播学"],
    ["翻译|外语|外国语|.*?语$", "外国语言文学"],
    ["法学|法律", "法学"],
    ["中文|汉语言", "汉语言"],
    ["哲学", "哲学"],
    ["金融", "金融类"],
    ["经济|贸易", "经济学类"],
    ["历史|文物|考古|文博", "历史学类"],
    ["政治学|思想政治", "政治学类"],
    ["工商管理", "工商管理"],
    ["心理", "心理学"],
    ["公共管理|行政管理|社会保障", "公共管理类"],
    ["社会学|社会工作|人类学|民族学|民俗学", "社会学类"],
    ["数学", "数学类"],
    ["电气", "电气类"],
    ["通信", "通信类"],
    ["电子", "电子类"],
    ["机械", "机械类"],
    ["计算机|人工智能", "计算机类"],
    ["软件", "软件工程"],
    ["土木", "土木工程类"],
    ["^统计学$|应用统计|经济统计", "统计学类"],
    ["建筑|城乡规划", "建筑学类"],
    ["生物", "生物类"],
    ["材料", "材料类"],
    ["化学", "化学类"],
    ["基地", "基地班"],
    ["拔尖", "拔尖班"],
    ["环境科学|环境工程", "环境科学类"],
    ["临床医学", "临床医学"],
    ["口腔", "口腔医学"],
    ["临床药学|药学", "药学类"],
    ["林学|林业|草|动物|水产|农业", "农业类"],
    ["信息管理|档案|图书", "信息管理与图书情报"],
    ["地球|地质", "地质学"]
]

# Convert the lists to dataframes
majorData = pd.DataFrame(majorList, columns=["noun", "major"])
majorData_rough = pd.DataFrame(majorList2, columns=["noun", "major"])

# Function to read and process data
def read_and_process_data(year):
    """
    读取并处理指定年份的山东省普通一批投档线数据。

    参数:
    year -- 数据的年份

    返回:
    dt -- 处理后的数据表，包含专业、院校、计划数和位次信息
    dt_school -- 经过处理的院校排名数据表
    """
    # 从Excel文件中读取数据，指定列类型为字符串，并去除所有列的空值
    dt = pd.read_excel(f"data/{year}年山东省普通一批投档线.xlsx", dtype=str).dropna(axis=1, how='all')

    # 重命名列
    dt.columns = ["专业", "院校", "计划数", "位次"]

    # 移除包含“定向”或“预科”的专业行
    dt = dt[~dt['专业'].str.contains("定向|预科")]

    # 移除院校和专业列中的括号内的信息
    dt[['院校', '专业']] = dt[['院校', '专业']].applymap(lambda x: re.sub(r"\(.*?\)|\{.*?\}|\[.*?\]", "", x))

    # 处理位次列，将“前50名”转换为数字50，并移除位次为0的行
    dt['位次'] = dt['位次'].fillna('0').astype(str).apply(lambda x: re.sub(r"前50名", "50", x) if x else '0').astype(int)
    dt = dt[dt['位次'] != 0]

    # 处理位次为正无穷或负无穷的行，并计算每个院校的位次中位数
    dt_school = dt[~dt['位次'].isin([np.inf, -np.inf])]
    dt_school = dt_school.groupby('院校').agg(rank_by_school=('位次', 'median')).reset_index()

    # 将院校的位次中位数转换为整数类型
    dt_school['rank_by_school'] = dt_school['rank_by_school'].astype(int)

    return dt, dt_school

In [ ]:
# Read data
years = range(2020, 2024)
data_list = {f"dt{year}": read_and_process_data(year) for year in years}
# Split data
for year in years:
    globals()[f"dt{year}"], globals()[f"dt{year}_school"] = data_list[f"dt{year}"]

In [ ]:
# Function to process data
def process_data(data):
    data['专业'] = data['专业'].str[2:]
    data = data.groupby(['院校', '专业']).agg({'计划数': 'sum', '位次': 'max'}).reset_index()
    return data

# Process data
dt2023_cmb_pd = process_data(dt2023)
dt2022_cmb_pd = process_data(dt2022)
dt2021_cmb_pd = process_data(dt2021)
dt2020_cmb_pd = process_data(dt2020)